In [1]:
# Jupyter options
%load_ext autoreload
%autoreload

# Some basic imports to use
import tensorflow as tf
import numpy as np
import random
from tensorflow import keras
import matplotlib.pyplot as plt

import argparse
import copy

from markov.s3_boto_data_store import S3BotoDataStore, S3BotoDataStoreParameters
from markov.sagemaker_graph_manager import get_graph_manager
import markov.defaults as defaults
from gym.envs.registration import register
import gym

from rl_coach.base_parameters import TaskParameters, Frameworks
from rl_coach.core_types import TrainingSteps, EnvironmentEpisodes, EnvironmentSteps, RunPhase
from rl_coach.utils import short_dynamic_import
import imp

import markov
from markov import utils
import markov.environments
#import markov.presets
import os
import sys
import math

# Just set some seeds so we are consistent across runs
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
#tf.random.set_random_seed(random_seed) # just so it is forward compatible

# Fix any missing access keys for MINIO
os.environ['S3_ENDPOINT_URL'] = 'http://172.17.0.1:9000'
os.environ['AWS_ACCESS_KEY_ID'] = 'minio'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'miniokey123'
os.environ['ROS_MASTER_URI'] = 'http://localhost:11311'

# Data loading

In [2]:
def add_items_to_dict(target_dict, source_dict):
    updated_task_parameters = copy.copy(source_dict)
    updated_task_parameters.update(target_dict)
    return updated_task_parameters

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

"""
Default reward function is the centerline.
"""
def reward_function(params):

    distance_from_center = params['distance_from_center']
    track_width = params['track_width']

    marker_1 = 0.1 * track_width
    marker_2 = 0.25 * track_width
    marker_3 = 0.5 * track_width

    reward = 1e-3
    if distance_from_center <= marker_1:
        reward = 1
    elif distance_from_center <= marker_2:
        reward = 0.5
    elif distance_from_center <= marker_3:
        reward = 0.1
    else:
        reward = 1e-3  # likely crashed/ close to off track

    return float(reward)

        
args_dict = {
"checkpoint_save_secs":  60*3,
"local_model_directory": "./checkpoint",
"num_workers": 1,
"model_s3_bucket": "deepracer",
"model_s3_prefix": "train",
"METRICS_S3_BUCKET": "deepracer",
"METRICS_S3_OBJECT_KEY": "train/metrics.json",
"aws_region": "us-east-1",
"save_frozen_graph": True,
}
args = objectview(args_dict)

# Register the gym enviroment, this will give clients the ability to creat the enviroment object
print(gym.envs.registration.registry)
register(id=defaults.ENV_ID, entry_point=defaults.ENTRY_POINT,
         max_episode_steps=defaults.MAX_STEPS, reward_threshold=defaults.THRESHOLD, kwargs={
             "reward_function": reward_function
         })



sm_hyperparams_dict = {}
graph_manager, robomaker_hyperparams_json = get_graph_manager(**sm_hyperparams_dict)

# TODO: support other frameworks
task_parameters = TaskParameters(framework_type=Frameworks.tensorflow,
                                 checkpoint_save_secs=args.checkpoint_save_secs)
task_parameters.__dict__['checkpoint_save_dir'] = args.local_model_directory
task_parameters.__dict__['experiment_path'] = args.local_model_directory + "/expirement"
task_parameters.__dict__ = add_items_to_dict(task_parameters.__dict__, args.__dict__)

data_store_params_instance = S3BotoDataStoreParameters(bucket_name=args.model_s3_bucket,
                                                       s3_folder=args.model_s3_prefix,
                                                       checkpoint_dir=args.local_model_directory,
                                                       aws_region=args.aws_region)
data_store = S3BotoDataStore(data_store_params_instance)

if args.save_frozen_graph:
    data_store.graph_manager = graph_manager

graph_manager.data_store_params = data_store_params_instance
graph_manager.data_store = data_store
#graph_manager.should_stop = should_stop_training_based_on_evaluation


Using the following hyper-parameters
{
  "batch_size": 64,
  "beta_entropy": 0.01,
  "discount_factor": 0.999,
  "e_greedy_value": 0.05,
  "epsilon_steps": 10000,
  "exploration_type": "categorical",
  "loss_type": "mean squared error",
  "lr": 0.0003,
  "num_episodes_between_training": 20,
  "num_epochs": 2,
  "stack_size": 1,
  "term_cond_avg_score": 100000.0,
  "term_cond_max_episodes": 100000
}


# Data Analysis

In [3]:
#print(dir(graph_manager))
graph_manager.create_graph(task_parameters)
#graph_manager.close()
#graph_manager.restore_checkpoint()r



Creating graph - name: BasicRLGraphManager


Loaded action space from file: [{'speed': 2, 'index': 0, 'steering_angle': -30}, {'speed': 2, 'index': 1, 'steering_angle': -15}, {'speed': 2, 'index': 2, 'steering_angle': 0}, {'speed': 2, 'index': 3, 'steering_angle': 15}, {'speed': 2, 'index': 4, 'steering_angle': 30}]
SIM_TRACE_LOG:0,0,6.1446,2.7064,-3.0716,0.00,0.00,0,1.0000,False,True,0.8063,1,20.51,1581228793.2681847

SIM_TRACE_LOG:0,1,6.1130,2.6914,-2.9905,-0.52,2.00,0,1.0000,False,True,0.9612,2,20.51,1581228793.337871



Creating agent - name: agent
Network: main, Copies: 2 (online network | target network)
----------------------------------------------------------
Sub-network for head: v_values_head
	Input Embedder: observation
		Input Normalization (scale = 255.0, offset = 0.0)
		Convolution (num filters = 32, kernel size = 8, stride = 4)
		Activation (type = <function relu at 0x7f4124085378>)
		Convolution (num filters = 64, kernel size = 4, stride = 2)
		Activation (type = <function relu at 0x7f4124085378>)
		Convolution (num filters = 64, kernel size = 3, stride = 1)
		Activation (type = <function relu at 0x7f4124085378>)
	Input Embedder: observation
		Input Normalization (scale = 255.0, offset = 0.0)
		Convolution (num filters = 32, kernel size = 8, stride = 4)
		Activation (type = <function relu at 0x7f4124085378>)
		Convolution (num filters = 64, kernel size = 4, stride = 2)
		Activation (type = <function relu at 0x7f4124085378>)
		Convolution (num filters = 64, kernel size = 3, stride = 1)
		A

In [ ]:
graph_manager.reset_internal_state()
with graph_manager.phase_context(RunPhase.TRAIN):
        last_checkpoint = 0
        act_steps = graph_manager.agent_params.algorithm.num_consecutive_playing_steps.num_steps

        for i in range(int(graph_manager.improve_steps.num_steps/act_steps)):
            try:
                # This will only work for DeepRacerRacetrackEnv enviroments
                graph_manager.top_level_manager.environment.env.env.set_allow_servo_step_signals(True)
            except Exception as ex:
                utils.json_format_logger("Method not defined in enviroment class: {}".format(ex),
                                   **utils.build_system_error_dict(utils.SIMAPP_SIMULATION_WORKER_EXCEPTION, utils.SIMAPP_EVENT_ERROR_CODE_500))

            if type(graph_manager.agent_params.algorithm.num_consecutive_playing_steps) == EnvironmentSteps:
                graph_manager.act(EnvironmentSteps(num_steps=act_steps), wait_for_full_episodes=graph_manager.agent_params.algorithm.act_for_full_episodes)
            elif type(graph_manager.agent_params.algorithm.num_consecutive_playing_steps) == EnvironmentEpisodes:
                graph_manager.act(EnvironmentEpisodes(num_steps=act_steps))

            try:
                # This will only work for DeepRacerRacetrackEnv enviroments
                graph_manager.top_level_manager.environment.env.env.set_allow_servo_step_signals(False)
                graph_manager.top_level_manager.environment.env.env.stop_car()
            except Exception as ex:
                utils.json_format_logger("Method not defined in enviroment class: {}".format(ex),
                                   **utils.build_system_error_dict(utils.SIMAPP_SIMULATION_WORKER_EXCEPTION, utils.SIMAPP_EVENT_ERROR_CODE_500))

SIM_TRACE_LOG:0,0,6.1479,2.7083,-3.0829,0.00,0.00,0,1.0000,False,True,0.7902,1,20.51,1581228803.8865378

SIM_TRACE_LOG:0,1,6.0936,2.6826,-2.9491,0.26,2.00,3,1.0000,False,True,1.0561,2,20.51,1581228803.9948025

SIM_TRACE_LOG:0,2,6.0749,2.6723,-2.9048,0.52,2.00,4,1.0000,False,True,1.1520,2,20.51,1581228804.0273194

SIM_TRACE_LOG:0,3,6.0411,2.6511,-2.8262,0.00,2.00,2,1.0000,False,True,1.3202,2,20.51,1581228804.0874033

SIM_TRACE_LOG:0,4,6.0073,2.6277,-2.7647,-0.26,2.00,1,0.5000,False,True,1.4881,3,20.51,1581228804.15404

SIM_TRACE_LOG:0,5,5.9729,2.6040,-2.7067,0.52,2.00,4,0.5000,False,True,1.6594,3,20.51,1581228804.2193942

SIM_TRACE_LOG:0,6,5.9394,2.5772,-2.6546,-0.26,2.00,1,0.5000,False,True,1.8478,3,20.51,1581228804.2884793

SIM_TRACE_LOG:0,7,5.9089,2.5528,-2.6099,0.52,2.00,4,0.5000,False,True,2.0039,4,20.51,1581228804.3522642

SIM_TRACE_LOG:0,8,5.8758,2.5238,-2.5595,-0.52,2.00,0,0.1000,False,True,2.1739,4,20.51,1581228804.422588

SIM_TRACE_LOG:0,9,5.8458,2.4977,-2.5276,0.52,2.00,4,0.1

reward: 6.80800001


SIM_TRACE_LOG:1,0,5.1693,2.5309,-2.8231,0.00,0.00,0,1.0000,False,True,0.6808,10,20.51,1581228806.0186086

SIM_TRACE_LOG:1,1,5.1635,2.5292,-2.8247,-0.52,2.00,0,1.0000,False,True,0.7101,10,20.51,1581228806.0898936



Training - Name: main_level/agent Worker: 0 Episode: 1 Total reward: 5.81 Steps: 24 Training iteration: 0 


SIM_TRACE_LOG:1,2,5.1448,2.5240,-2.8311,0.00,2.00,2,1.0000,False,True,0.8047,10,20.51,1581228806.1575863

SIM_TRACE_LOG:1,3,5.1186,2.5174,-2.8419,-0.26,2.00,1,1.0000,False,True,0.9366,11,20.51,1581228806.2268322

SIM_TRACE_LOG:1,4,5.0829,2.5088,-2.8563,0.52,2.00,4,1.0000,False,True,1.1155,11,20.51,1581228806.290066

SIM_TRACE_LOG:1,5,5.0433,2.4974,-2.8575,0.52,2.00,4,1.0000,False,True,1.3154,11,20.51,1581228806.359218

SIM_TRACE_LOG:1,6,5.0068,2.4858,-2.8517,0.26,2.00,3,1.0000,False,True,1.5015,12,20.51,1581228806.4230742

SIM_TRACE_LOG:1,7,4.9668,2.4727,-2.8452,0.00,2.00,2,1.0000,False,True,1.7007,12,20.51,1581228806.4931936

SIM_TRACE_LOG:1,8,4.9298,2.4617,-2.8467,-0.26,2.00,1,1.0000,False,True,1.8860,12,20.51,1581228806.5573258

SIM_TRACE_LOG:1,9,4.8914,2.4522,-2.8593,-0.52,2.00,0,1.0000,False,True,2.0775,13,20.51,1581228806.623427

SIM_TRACE_LOG:1,10,4.8534,2.4443,-2.8785,0.52,2.00,4,1.0000,False,True,2.2514,13,20.51,1581228806.6911113

SIM_TRACE_LOG:1,11,4.8139,2.4353,-2.8879,0.26

...

SIM_TRACE_LOG:198,0,7.9023,1.9490,2.4982,0.00,0.00,0,1.0000,False,True,0.6796,163,20.51,1581229428.0530372

SIM_TRACE_LOG:198,1,7.8996,1.9507,2.4998,0.52,2.00,4,1.0000,False,True,0.6947,163,20.51,1581229428.1210492

SIM_TRACE_LOG:198,2,7.8832,1.9607,2.5125,-0.26,2.00,1,1.0000,False,True,0.7861,163,20.51,1581229428.1856534

SIM_TRACE_LOG:198,3,7.8540,1.9784,2.5297,0.26,2.00,3,1.0000,False,True,0.9487,164,20.51,1581229428.2538447

SIM_TRACE_LOG:198,4,7.8218,1.9975,2.5465,-0.52,2.00,0,1.0000,False,True,1.1265,164,20.51,1581229428.319034

SIM_TRACE_LOG:198,5,7.7854,2.0203,2.5553,0.00,2.00,2,1.0000,False,True,1.3084,164,20.51,1581229428.3872879

SIM_TRACE_LOG:198,6,7.7483,2.0422,2.5656,0.26,2.00,3,1.0000,False,True,1.5044,165,20.51,1581229428.4560091

SIM_TRACE_LOG:198,7,7.7141,2.0619,2.5788,-0.52,2.00,0,0.5000,False,True,1.6838,165,20.51,1581229428.5224848

SIM_TRACE_LOG:198,8,7.6797,2.0833,2.5798,-0.26,2.00,1,0.5000,False,True,1.8347,165,20.51,1581229428.5874753

SIM_TRACE_LOG:198,9,7.644

reward: 26.706000010000018
Training - Name: main_level/agent Worker: 0 Episode: 199 Total reward: 25.71 Steps: 8070 Training iteration: 0 


SIM_TRACE_LOG:199,0,7.1980,2.6743,2.5960,0.00,0.00,0,1.0000,False,True,0.6665,172,20.51,1581229433.260627

SIM_TRACE_LOG:199,1,7.1942,2.6770,2.5938,-0.52,2.00,0,1.0000,False,True,0.6882,172,20.51,1581229433.325005

SIM_TRACE_LOG:199,2,7.1785,2.6884,2.5829,0.52,2.00,4,1.0000,False,True,0.7775,172,20.51,1581229433.3937092

SIM_TRACE_LOG:199,3,7.1500,2.7075,2.5761,0.26,2.00,3,1.0000,False,True,0.9378,173,20.51,1581229433.466247

SIM_TRACE_LOG:199,4,7.1166,2.7300,2.5700,0.00,2.00,2,1.0000,False,True,1.1086,173,20.51,1581229433.5298822

SIM_TRACE_LOG:199,5,7.0848,2.7516,2.5638,0.00,2.00,2,1.0000,False,True,1.2713,173,20.51,1581229433.594343

SIM_TRACE_LOG:199,6,7.0518,2.7735,2.5618,0.26,2.00,3,0.5000,False,True,1.4495,174,20.51,1581229433.6626334

SIM_TRACE_LOG:199,7,7.0109,2.7988,2.5689,0.26,2.00,3,0.5000,False,True,1.6634,174,20.51,1581229433.7419465

SIM_TRACE_LOG:199,8,6.9795,2.8178,2.5745,-0.52,2.00,0,0.5000,False,True,1.7879,174,20.51,1581229433.7961552

SIM_TRACE_LOG:199,9,6.9466,2.8

reward: 9.306000009999995
Training - Name: main_level/agent Worker: 0 Episode: 200 Total reward: 8.31 Steps: 8095 Training iteration: 0 


SIM_TRACE_LOG:200,0,6.1696,2.7165,-3.1351,0.00,0.00,0,1.0000,False,True,0.6836,1,20.51,1581229435.5301056

SIM_TRACE_LOG:200,1,6.1658,2.7163,-3.1344,0.26,2.00,3,1.0000,False,True,0.7023,1,20.51,1581229435.5966237

SIM_TRACE_LOG:200,2,6.1447,2.7143,-3.1225,0.52,2.00,4,1.0000,False,True,0.8051,1,20.51,1581229435.6659744

SIM_TRACE_LOG:200,3,6.1177,2.7106,-3.1013,0.52,2.00,4,1.0000,False,True,0.9370,2,20.51,1581229435.7337537

SIM_TRACE_LOG:200,4,6.0811,2.7041,-3.0696,0.00,2.00,2,1.0000,False,True,1.1170,2,20.51,1581229435.7992184

SIM_TRACE_LOG:200,5,6.0419,2.6974,-3.0476,-0.26,2.00,1,1.0000,False,True,1.3090,2,20.51,1581229435.8646803

SIM_TRACE_LOG:200,6,6.0049,2.6912,-3.0313,0.52,2.00,4,1.0000,False,True,1.4904,3,20.51,1581229435.9307017

SIM_TRACE_LOG:200,7,5.9619,2.6820,-3.0036,0.26,2.00,3,1.0000,False,True,1.7061,3,20.51,1581229436.0026643

SIM_TRACE_LOG:200,8,5.9252,2.6732,-2.9808,0.26,2.00,3,1.0000,False,True,1.8871,3,20.51,1581229436.0676992

SIM_TRACE_LOG:200,9,5.8877,2.6635,-2

# Create Model

In [ ]:
graph_manager.save_checkpoint()

# Model Performance

# Model Analysis